In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import sqlalchemy
from sqlalchemy import create_engine # database connection
import sqlite3

from IPython.display import display, clear_output
from sklearn.naive_bayes import MultinomialNB
from datetime import datetime

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
# print(check_output([]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [2]:
# Parameters to change depending on season and wek we are running for
this_season = '2017/2018'
this_week = 18

In [3]:
# engine  = create_engine("sqlite:///../input/database.sqlite")
with sqlite3.connect('database.sqlite') as engine:
#    matches = pd.read_sql_query('SELECT * FROM Match where league_id = 1729 and season in ("2010/2011", "2011/2012", "2012/2013", "2013/2014", "2014/2015", "2015/2016");'
#                                          , engine)
    matches = pd.read_sql_query('SELECT * FROM Match where league_id = 1729 ;', engine)


matches = matches[matches.columns[:11]]
teams = pd.read_sql_query('SELECT * FROM Team;', engine)

# Add team names & tidy up
matches = pd.merge(left=matches, right=teams, how='left', left_on='home_team_api_id', right_on='team_api_id')
matches = matches.drop(['country_id','league_id', 'home_team_api_id', 'id_y', 'team_api_id', 'team_short_name'], axis=1)
matches.rename(columns={'id_x':'match_id', 'team_long_name':'home_team'}, inplace=True)
matches = pd.merge(left=matches, right=teams, how='left', left_on='away_team_api_id', right_on='team_api_id')
matches = matches.drop(['id', 'match_api_id', 'away_team_api_id','team_api_id', 'team_short_name'], axis=1)
matches.rename(columns={'id_x':'match_id', 'team_long_name':'away_team'}, inplace=True)

# matches.tail()
matches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3040 entries, 0 to 3039
Data columns (total 10 columns):
match_id              3040 non-null int64
season                3040 non-null object
stage                 3040 non-null int64
date                  3040 non-null object
home_team_goal        3040 non-null int64
away_team_goal        3040 non-null int64
team_fifa_api_id_x    3040 non-null float64
home_team             3040 non-null object
team_fifa_api_id_y    3040 non-null float64
away_team             3040 non-null object
dtypes: float64(2), int64(4), object(4)
memory usage: 261.2+ KB


In [4]:
def convert_date_to_code16(matches_data_16):
    date = []
    for i in matches_data_16['date']:
        date.append(i)

    x = date[0]
    d = datetime.strptime(x, "%d/%m/%y").date()
    dict2 = {}
    base = 42595
    for i in date:
        date_l = datetime.strptime(i, "%d/%m/%y").date()
        date_dif = date_l - d
        dict2[date_l] = str(base + date_dif.days)
    t = []
    for i in range(len(date)):
        t.append(dict2[datetime.strptime(date[i], '%d/%m/%y').date()])
    matches_data_16['date'] = list(map((lambda i: t[i]), range(len(t))))
    return matches_data_16

def convert_date_to_code17(matches_data_17):
    stage = []
    for i in matches_data_17['stage']:
        stage.append(i)
    date = []
    for i in matches_data_17['date']:
        date.append(i)
    dict_date = {}
    for i, j in zip(stage, date):
        dict_date[i] = datetime.strptime(j, "%d/%m/%y").date()
    dict2 = {}
    base = 42959
    for i in stage:
        if i == 1:
            dict2[i] = base
        else:
            date_dif = dict_date[i] - dict_date[i - 1]
            if date_dif.days < 14:
                dict2[i] = dict2[i - 1] + 7
            if date_dif.days >= 14:
                dict2[i] = dict2[i - 1] + 14
    t = []
    for i in stage:
        t.append(str(dict2[i]))
    matches_data_17['date'] = list(map((lambda i: t[i]), range(len(t))))
    return matches_data_17

def convert_team_name(last_season):
    for index, row in last_season.iterrows():
        if row['home_team'] == 'Man United':
            last_season.loc[index, 'home_team'] = 'Manchester United'
        if row['home_team'] == 'Man City':
            last_season.loc[index, 'home_team'] = 'Manchester City'
        if row['home_team'] == 'Leicester':
            last_season.loc[index, 'home_team'] = 'Leicester City'
        if row['home_team'] == 'West Brom':
            last_season.loc[index, 'home_team'] = 'West Bromwich Albion'
        if row['home_team'] == 'Newcastle':
            last_season.loc[index, 'home_team'] = 'Newcastle United'
        if row['home_team'] == 'Stoke':
            last_season.loc[index, 'home_team'] = 'Stoke City'
        if row['home_team'] == 'Swansea':
            last_season.loc[index, 'home_team'] = 'Swansea City'
        if row['home_team'] == 'West Ham':
            last_season.loc[index, 'home_team'] = 'West Ham United'
        if row['home_team'] == 'Tottenham':
            last_season.loc[index, 'home_team'] = 'Tottenham Hotspur'
        if row['home_team'] == 'Hull':
            last_season.loc[index, 'home_team'] = 'Hull City'
        
        if row['away_team'] == 'Man United':
            last_season.loc[index, 'away_team'] = 'Manchester United'
        if row['away_team'] == 'Man City':
            last_season.loc[index, 'away_team'] = 'Manchester City'
        if row['away_team'] == 'Leicester':
            last_season.loc[index, 'away_team'] = 'Leicester City'
        if row['away_team'] == 'West Brom':
            last_season.loc[index, 'away_team'] = 'West Bromwich Albion'
        if row['away_team'] == 'Newcastle':
            last_season.loc[index, 'away_team'] = 'Newcastle United'
        if row['away_team'] == 'Stoke':
            last_season.loc[index, 'away_team'] = 'Stoke City'
        if row['away_team'] == 'Swansea':
            last_season.loc[index, 'away_team'] = 'Swansea City'
        if row['away_team'] == 'West Ham':
            last_season.loc[index, 'away_team'] = 'West Ham United'
        if row['away_team'] == 'Tottenham':
            last_season.loc[index, 'away_team'] = 'Tottenham Hotspur'
        if row['away_team'] == 'Hull':
            last_season.loc[index, 'away_team'] = 'Hull City'
    return last_season
    

with sqlite3.connect('database2.sqlite') as con:
    matches_data_16 = pd.read_sql('select * from match;', con)
    matches_data_17 = pd.read_sql('select * from match17;', con)
# matches_data_16['date'] = pd.to_datetime(matches_data_16['date'])

convert_date_to_code16(matches_data_16)
convert_date_to_code17(matches_data_17)
# latest_matches = pd.concat([matches_data_16, matches_data_17])
# latest_matches = latest_matches.reset_index(drop=True)
# latest_matches.info()

# Add to full training data to predict current season
matches = pd.concat([matches, matches_data_16, matches_data_17])
matches = matches.reset_index(drop=True)
# matches.info()

# Create a full set of match data that can be used with feature engineering later
full_matches = matches.copy()
# full_matches.tail(100)
# full_matches.info()

In [5]:
full_matches.drop(matches[matches.season == '2008/2009'].index, inplace=True)
full_matches.drop(matches[matches.season == '2009/2010'].index, inplace=True)
full_matches.drop(matches[matches.season == '2010/2011'].index, inplace=True)
full_matches.drop(matches[matches.season == '2011/2012'].index, inplace=True)
full_matches

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y
1520,Sunderland,0,2012-08-18 00:00:00,Arsenal,0,3249,2012/2013,1,1.0,106.0
1521,Manchester United,0,2012-08-20 00:00:00,Everton,1,3250,2012/2013,1,7.0,11.0
1522,Norwich City,0,2012-08-18 00:00:00,Fulham,5,3251,2012/2013,1,144.0,1792.0
1523,Southampton,2,2012-08-19 00:00:00,Manchester City,3,3252,2012/2013,1,10.0,17.0
1524,Tottenham Hotspur,1,2012-08-18 00:00:00,Newcastle United,2,3253,2012/2013,1,13.0,18.0
1525,Swansea City,5,2012-08-18 00:00:00,Queens Park Rangers,0,3254,2012/2013,1,15.0,1960.0
1526,Stoke City,1,2012-08-18 00:00:00,Reading,1,3255,2012/2013,1,1793.0,1806.0
1527,Liverpool,0,2012-08-18 00:00:00,West Bromwich Albion,3,3256,2012/2013,1,109.0,9.0
1528,Aston Villa,0,2012-08-18 00:00:00,West Ham United,1,3257,2012/2013,1,19.0,2.0
1529,Chelsea,2,2012-08-19 00:00:00,Wigan Athletic,0,3258,2012/2013,1,1917.0,5.0


In [6]:
# State the result as home or away win/draw/lose - 6 possibilites
# Create a binary result
def determine_result(match_list):
    match_list['home_win'] = np.where(match_list['home_team_goal'] > match_list['away_team_goal'], 1, 0)
    match_list['home_draw'] = np.where(match_list['home_team_goal'] == match_list['away_team_goal'], 1, 0)
    match_list['home_lose'] = np.where(match_list['home_team_goal'] < match_list['away_team_goal'], 1, 0)
    match_list['away_win'] = np.where(match_list['home_team_goal'] < match_list['away_team_goal'], 1, 0)
    match_list['away_draw'] = np.where(match_list['home_team_goal'] == match_list['away_team_goal'], 1, 0)
    match_list['away_lose'] = np.where(match_list['home_team_goal'] > match_list['away_team_goal'], 1, 0)

# Set up the matches data how I need it

# Add binary feature for W/D/L home and away
determine_result(full_matches)

# Sort in date order
full_matches.sort_values(by='date', inplace=True)

full_matches.head()

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y,home_win,home_draw,home_lose,away_win,away_draw,away_lose
1520,Sunderland,0,2012-08-18 00:00:00,Arsenal,0,3249,2012/2013,1,1.0,106.0,0,1,0,0,1,0
1527,Liverpool,0,2012-08-18 00:00:00,West Bromwich Albion,3,3256,2012/2013,1,109.0,9.0,1,0,0,0,0,1
1526,Stoke City,1,2012-08-18 00:00:00,Reading,1,3255,2012/2013,1,1793.0,1806.0,0,1,0,0,1,0
1525,Swansea City,5,2012-08-18 00:00:00,Queens Park Rangers,0,3254,2012/2013,1,15.0,1960.0,0,0,1,1,0,0
1528,Aston Villa,0,2012-08-18 00:00:00,West Ham United,1,3257,2012/2013,1,19.0,2.0,1,0,0,0,0,1


In [7]:
# Cope with newly promoted teams with limited or no stats
team_data = {'team':['West Bromwich Albion', 'Stoke City', 'Hull City',
                     'Wolverhampton Wanderers', 'Birmingham City', 'Burnley',
                     'Newcastle United', 'West Bromwich Albion', 'Blackpool',
                     'Queens Park Rangers', 'Norwich City', 'Swansea City',
                     'Reading', 'Southampton', 'West Ham United',
                     'Cardiff City', 'Crystal Palace', 'Hull City',
                     'Leicester City', 'Burnley', 'Queens Park Rangers',
                     'Bournemouth', 'Watford', 'Norwich City',
                     'Burnley', 'Middlesbrough', 'Hull City',
                     'Brighton', 'Newcastle United', 'Huddersfield'
                    ],
             'season':["2008/2009", "2008/2009", "2008/2009",
                       "2009/2010", "2009/2010", "2009/2010",
                       "2010/2011", "2010/2011", "2010/2011",
                       "2011/2012", "2011/2012", "2011/2012",
                       "2012/2013", "2012/2013", "2012/2013",
                       "2013/2014", "2013/2014", "2013/2014",
                       "2014/2015", "2014/2015", "2014/2015",
                       "2015/2016", "2015/2016", "2015/2016",
                       "2016/2017", "2016/2017", "2016/2017",
                       "2017/2018", "2017/2018", "2017/2018"
                      ]
            }
new_teams = pd.DataFrame(team_data, columns=['team', 'season'])
new_teams.head()

,team,season
0,West Bromwich Albion,2008/2009
1,Stoke City,2008/2009
2,Hull City,2008/2009
3,Wolverhampton Wanderers,2009/2010
4,Birmingham City,2009/2010


In [8]:
# Cater for new teams by setting the new team for that season to a generic name

for index, row in new_teams.iterrows():
    for index1, row1 in full_matches.iterrows():
        if (row1['home_team'] == row['team']) & (row1['season'] == row['season']):
            full_matches.loc[index1, 'home_team'] = 'Promoted'
        if (row1['away_team'] == row['team']) & (row1['season'] == row['season']):
            full_matches.loc[index1, 'away_team'] = 'Promoted'

full_matches.head(20)

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y,home_win,home_draw,home_lose,away_win,away_draw,away_lose
1520,Sunderland,0,2012-08-18 00:00:00,Arsenal,0,3249,2012/2013,1,1.0,106.0,0,1,0,0,1,0
1527,Liverpool,0,2012-08-18 00:00:00,West Bromwich Albion,3,3256,2012/2013,1,109.0,9.0,1,0,0,0,0,1
1526,Stoke City,1,2012-08-18 00:00:00,Promoted,1,3255,2012/2013,1,1793.0,1806.0,0,1,0,0,1,0
1525,Swansea City,5,2012-08-18 00:00:00,Queens Park Rangers,0,3254,2012/2013,1,15.0,1960.0,0,0,1,1,0,0
1528,Aston Villa,0,2012-08-18 00:00:00,Promoted,1,3257,2012/2013,1,19.0,2.0,1,0,0,0,0,1
1522,Norwich City,0,2012-08-18 00:00:00,Fulham,5,3251,2012/2013,1,144.0,1792.0,1,0,0,0,0,1
1524,Tottenham Hotspur,1,2012-08-18 00:00:00,Newcastle United,2,3253,2012/2013,1,13.0,18.0,1,0,0,0,0,1
1523,Promoted,2,2012-08-19 00:00:00,Manchester City,3,3252,2012/2013,1,10.0,17.0,1,0,0,0,0,1
1529,Chelsea,2,2012-08-19 00:00:00,Wigan Athletic,0,3258,2012/2013,1,1917.0,5.0,0,0,1,1,0,0
1521,Manchester United,0,2012-08-20 00:00:00,Everton,1,3250,2012/2013,1,7.0,11.0,1,0,0,0,0,1


In [9]:
# Convert home & team into a binary feature, ie Arsenal_h or Arsenal_a
# Need all seasons data for team binary feature
full_match_features = pd.DataFrame(full_matches[['season', 'stage']])
                                   #columns=[['season', 'stage']])

full_match_features = pd.concat([full_match_features, pd.get_dummies(full_matches['home_team']).rename(columns=lambda x: str(x) + '_h')],
                                axis=1)
full_match_features = pd.concat([full_match_features, pd.get_dummies(full_matches['away_team']).rename(columns=lambda x: str(x) + '_a')],
                         axis=1)

full_match_features = full_match_features.reset_index(drop=True)
full_match_features.head()

,season,stage,Arsenal_h,Aston Villa_h,Bournemouth_h,Burnley_h,Chelsea_h,Crystal Palace_h,Everton_h,Fulham_h,...,Swansea_a,Swansea City_a,Tottenham_a,Tottenham Hotspur_a,Watford_a,West Brom_a,West Bromwich Albion_a,West Ham_a,West Ham United_a,Wigan Athletic_a
0,2012/2013,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2012/2013,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2012/2013,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2012/2013,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,2012/2013,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# To predict this season (1, this week only, remove this week from training set
train_match_features = full_match_features.loc[(full_match_features['season'] != this_season) | 
                                              (full_match_features['season'] == this_season) &
                                                  (full_match_features['stage'] < this_week)].copy()

train_match_features.drop(['season'], axis=1, inplace=True)
train_match_features.tail()

,stage,Arsenal_h,Aston Villa_h,Bournemouth_h,Burnley_h,Chelsea_h,Crystal Palace_h,Everton_h,Fulham_h,Hull_h,...,Swansea_a,Swansea City_a,Tottenham_a,Tottenham Hotspur_a,Watford_a,West Brom_a,West Bromwich Albion_a,West Ham_a,West Ham United_a,Wigan Athletic_a
2065,17,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2066,17,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2067,17,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2068,17,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2069,17,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Get the target results for training

# Function to determine whether result is a win/draw/lose
def determine_home_result(match):
    if match['home_team_goal'] > match['away_team_goal']:
        return 'win'
    elif match['home_team_goal'] < match['away_team_goal']:
        return 'lose'
    else:
        return 'draw'


#  Add the home team result column to the matches dataframe
full_matches['home_team_result'] = full_matches.apply(determine_home_result, axis=1)

# To predict this season, this week, remove latest week from training results
train_matches = full_matches.loc[(full_matches['season'] != this_season) | 
                                              (full_matches['season'] == this_season) &
                                                  (full_matches['stage'] < this_week)].copy()

targets = train_matches['home_team_result'].values
train_matches.tail(10)

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y,home_win,home_draw,home_lose,away_win,away_draw,away_lose,home_team_result
3588,Promoted,0,43092,Tottenham,2,7168,2017/2018,17,NaN,NaN,1,0,0,0,0,1,win
3587,Man City,4,43092,Swansea,0,7167,2017/2018,17,NaN,NaN,0,0,1,1,0,0,lose
3586,Leicester,4,43092,Southampton,1,7166,2017/2018,17,NaN,NaN,0,0,1,1,0,0,lose
3585,Everton,1,43092,Newcastle,0,7165,2017/2018,17,NaN,NaN,0,0,1,1,0,0,lose
3589,Arsenal,0,43092,West Ham,0,7169,2017/2018,17,NaN,NaN,0,1,0,0,1,0,draw
3583,West Brom,0,43092,Liverpool,0,7163,2017/2018,17,NaN,NaN,0,1,0,0,1,0,draw
3582,Chelsea,3,43092,Promoted,1,7162,2017/2018,17,NaN,NaN,0,0,1,1,0,0,lose
3581,Watford,1,43092,Crystal Palace,2,7161,2017/2018,17,NaN,NaN,1,0,0,0,0,1,win
3580,Stoke,0,43092,Burnley,1,7160,2017/2018,17,NaN,NaN,1,0,0,0,0,1,win
3584,Bournemouth,0,43092,Man United,1,7164,2017/2018,17,NaN,NaN,1,0,0,0,0,1,win


In [12]:
# Get the test matches in correct format:

# Predict this season this week
test_match_features = full_match_features.loc[(full_match_features['season'] == this_season) &
                                              (full_match_features['stage'] == this_week)].copy()

test_match_features.drop(['season'], axis=1, inplace=True)
test_match_features

,stage,Arsenal_h,Aston Villa_h,Bournemouth_h,Burnley_h,Chelsea_h,Crystal Palace_h,Everton_h,Fulham_h,Hull_h,...,Swansea_a,Swansea City_a,Tottenham_a,Tottenham Hotspur_a,Watford_a,West Brom_a,West Bromwich Albion_a,West Ham_a,West Ham United_a,Wigan Athletic_a
2070,18,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2071,18,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2072,18,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2073,18,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2074,18,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2075,18,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2076,18,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2077,18,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2078,18,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2079,18,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Don't have the target results yet but I have entered dummy data of 0-0 draws. 
# Then re-run with actual scores after they are played for comparison
model_test_matches = full_matches.loc[(full_matches['season'] == this_season) & 
                                     (full_matches['stage'] == this_week)].copy()

model_test_matches = model_test_matches.reset_index(drop=True)
model_test_matches

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y,home_win,home_draw,home_lose,away_win,away_draw,away_lose,home_team_result
0,Swansea,1,43099,Everton,3,7179,2017/2018,18,NaN,NaN,1,0,0,0,0,1,win
1,Man United,2,43099,West Brom,1,7178,2017/2018,18,NaN,NaN,0,0,1,1,0,0,lose
2,Liverpool,4,43099,Bournemouth,0,7177,2017/2018,18,NaN,NaN,0,0,1,1,0,0,lose
3,Promoted,4,43099,Watford,1,7176,2017/2018,18,NaN,NaN,0,0,1,1,0,0,lose
4,West Ham,3,43099,Stoke,0,7175,2017/2018,18,NaN,NaN,0,0,1,1,0,0,lose
5,Tottenham,1,43099,Man City,4,7174,2017/2018,18,NaN,NaN,1,0,0,0,0,1,win
6,Southampton,0,43099,Chelsea,1,7172,2017/2018,18,NaN,NaN,1,0,0,0,0,1,win
7,Burnley,0,43099,Promoted,0,7171,2017/2018,18,NaN,NaN,0,1,0,0,1,0,draw
8,Newcastle,0,43099,Arsenal,1,7170,2017/2018,18,NaN,NaN,1,0,0,0,0,1,win
9,Crystal Palace,3,43099,Leicester,0,7173,2017/2018,18,NaN,NaN,0,0,1,1,0,0,lose


In [14]:
# Function to determine whether the highest prediction is for win/draw/lose

def predict_home_result(match):
    if (match['win'] >= match['draw']) & (match['win'] >= match['lose']):
        return 'win' # Favour a home win if probability equal
    elif (match['lose'] > match['win']) & (match['lose'] > match['draw']):
        return 'lose'
    else:
        return 'draw'

In [15]:
# Train, then predict 
model = MultinomialNB()

model.fit(train_match_features.values, targets)
predicted = model.predict_proba(test_match_features.values)

# Format the output into a DF with columns
predicted_table = pd.DataFrame(predicted,columns=['draw', 'lose', 'win'])

# Compare predicted with test actual results 
predicted_table['predict_res'] = predicted_table.apply(predict_home_result, axis=1)
predicted_table['actual_res'] = model_test_matches['home_team_result']

# Straight comparison - count of equal / total to get %
(predicted_table[predicted_table['predict_res'] 
                 == model_test_matches['home_team_result']].count()) / model_test_matches['home_team_result'].count()

draw           0.6
lose           0.6
win            0.6
predict_res    0.6
actual_res     0.6
dtype: float64

In [16]:
model.score(train_match_features.values, targets)

0.5661835748792271

In [17]:
compare_results = model_test_matches[['match_id', 'stage', 'home_team_goal', 
                                    'away_team_goal', 'home_team', 'away_team']].copy()
compare_results.rename(columns={'home_team_goal':'h_goal', 'away_team_goal':'a_goal'}, inplace=True)
compare_results = pd.concat([compare_results, predicted_table], axis=1)
compare_results

,match_id,stage,h_goal,a_goal,home_team,away_team,draw,lose,win,predict_res,actual_res
0,7179,18,3,1,Everton,Swansea,0.148160,0.133521,0.718319,win,win
1,7178,18,1,2,West Brom,Man United,0.257142,0.562811,0.180047,lose,lose
2,7177,18,0,4,Bournemouth,Liverpool,0.221378,0.525040,0.253582,lose,lose
3,7176,18,1,4,Watford,Promoted,0.213617,0.217622,0.568761,win,lose
4,7175,18,0,3,Stoke,West Ham,0.345615,0.217342,0.437042,win,lose
5,7174,18,4,1,Man City,Tottenham,0.390814,0.129027,0.480158,win,win
6,7172,18,1,0,Chelsea,Southampton,0.212810,0.141017,0.646173,win,win
7,7171,18,0,0,Promoted,Burnley,0.280488,0.532387,0.187125,lose,draw
8,7170,18,1,0,Arsenal,Newcastle,0.205662,0.065791,0.728547,win,win
9,7173,18,0,3,Leicester,Crystal Palace,0.206190,0.277156,0.516654,win,lose
